## Validação cruzada do modelo 1


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
livro = pd.read_csv('data/livro.csv')
livro

,intern_clock,1,2,3,4,x,y
0,31032,0.0,0.0,0.0,0.0,25.228563,-20.309231
1,31130,0.0,0.0,0.0,0.0,20.030073,-18.322621
2,31227,0.0,0.0,0.0,0.0,21.478640,-22.584932
3,31326,0.0,0.0,0.0,0.0,22.267633,-24.549507
4,31424,0.0,0.0,0.0,0.0,16.959957,-19.912392
...,...,...,...,...,...,...,...
699,99546,0.0,0.0,0.0,0.0,12.197288,31.348506
700,99643,0.0,0.0,0.0,0.0,12.502285,32.553389
701,99741,0.0,0.0,0.0,0.0,11.871969,33.459720
702,99840,0.0,0.0,0.0,0.0,11.883116,32.952679


In [2]:
# pra fazer os negócios funcionar
import nest_asyncio
nest_asyncio.apply()

# pra sumir com os warnings
import warnings
warnings.filterwarnings('ignore')  # não deu certo :(

In [59]:
from math import floor
from math import ceil

# tem que dar um shufle
livro = livro.sample(frac = 1)
size = livro.shape[0]

X_train = livro[['1', '2', '3', '4']][0:floor(size*70/100)].reset_index().drop('index', axis=1)
X_test = livro[['1', '2', '3', '4']][0:ceil(size*30/100)].reset_index().drop('index', axis=1)

x_train = livro['x'][0:floor(size*70/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*70/100),)
x_test = livro['x'][0:floor(size*30/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*30/100),)

y_train = livro['y'][0:floor(size*70/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*70/100),)
y_test = livro['y'][0:floor(size*30/100)].reset_index().drop('index', axis=1).to_numpy().reshape(floor(size*30/100),)

Treinando o modelo para x

In [63]:
# From: Bayesian Models for Astrophysical Data, Cambridge Univ. Press
# (c) 2017,  Joseph M. Hilbe, Rafael S. de Souza and Emille E. O. Ishida 
# 
# you are kindly asked to include the complete citation if you used this 
# material in a publication

# Code 4.7 - Multivariate normal linear model in Python using Stan
# 1 response (y) and 2 explanatory variables (x1, x2)

# ^^ a implementação foi inspirada no autor acima

import numpy as np
import statsmodels.api as sm
import stan
from scipy.stats import uniform, norm
import json
# Data
np.random.seed(1056)                 # set seed to replicate example
nobs= X_train.shape[0]                         # number of obs in model 
x1 = X_train['1'].to_numpy()          # random uniform variable
x2 = X_train['2'].to_numpy()          # second explanatory
x3 = X_train['3'].to_numpy()
x4 = X_train['4'].to_numpy()

X = np.column_stack((x1,x2, x3, x4))        # create response matrix
# X = np.column_stack((x1, x2))
X = sm.add_constant(X)              # add intercept

y = x_train # agora faz regressão na abcissa do centro de massa calculado do corpo de prova

# Fit
data = {}                                # build data dictionary
data['nobs'] = nobs                      # sample size
data['x'] = X                            # explanatory variable         
data['y'] = y                            # response variable
data['k'] = data['x'].shape[1]       # number of explanatory variables

# data = json.dumps(list(data))

# data = data.to_json()


# STAN code
stan_code = """
data {
    int<lower=1> k;  
    int<lower=0> nobs;                                 
    matrix[nobs, k] x;                     
    vector[nobs] y;                     
}
parameters {
    matrix[k,1] beta;                                             
    real<lower=0> sigma;               
}
transformed parameters{
    matrix[nobs,1] mu;
    vector[nobs] mu2;

    mu = x * beta;
    mu2 = to_vector(mu);                 // normal distribution 
                                         // does not take matrices as input
}
model {
    for (i in 1:k){                      // Diffuse normal priors for predictors
        beta[i] ~ normal(0.0, 100);
    }
    sigma ~ gamma(32.168426055, 49.11855);           // A variância dos sensores nao é baixa mas eles são relativamente confiáveis
    
    y ~ normal(mu2, sigma);              // Likelihood function
}
"""

#  sigma ~ gamma(76.083434761, 75.539775);           




# Compile model
model = stan.build(stan_code, data)

# perform fit
fit1 = model.sample(num_chains=4, num_samples=1000)


# Output
nlines = 9                                   # number of lines in screen output

output = str(fit1).split('\n')
for item in output[:nlines]:
    print(item)   

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_6jv2fi4x/model_hpnrszw7.stan', line 24, column 32: Argument
    49.11855 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_6jv2fi4x/model_hpnrszw7.stan', line 24, column 18: Argument
    32.168426055 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_6jv2fi4x/model_hpnrszw7.stan', line 22, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:  25% (2003/8000)
Sampling:  50% (4002/8000)
Sampling:  75% (6001/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received

<stan.Fit>
Parameters:
    beta: (5, 1)
    sigma: ()
    mu: (492, 1)
    mu2: (492,)
Draws: 4000


In [102]:
import arviz as az

summ = az.summary(fit1)
summ = summ[:][0:6]
summ

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"beta[0, 0]",14.241,3.383,8.054,20.600,0.049,0.035,4728.0,3285.0,1.0
"beta[1, 0]",-89.197,12.951,-114.015,-65.201,0.181,0.128,5169.0,2607.0,1.0
"beta[2, 0]",-8.227,11.971,-30.991,14.331,0.172,0.155,4827.0,3617.0,1.0
"beta[3, 0]",2.938,102.073,-176.022,204.999,1.439,1.585,5002.0,3182.0,1.0
"beta[4, 0]",-0.641,101.705,-183.506,193.055,1.429,1.508,5069.0,3275.0,1.0
sigma,71.455,0.663,70.131,72.651,0.009,0.006,5635.0,3087.0,1.0


In [105]:
coefsx = summ['mean'][0:6]
coefsx

beta[0, 0]    14.241
beta[1, 0]   -89.197
beta[2, 0]    -8.227
beta[3, 0]     2.938
beta[4, 0]    -0.641
sigma         71.455
Name: mean, dtype: float64

In [ ]:
Treinando o modelo para y

In [66]:
# From: Bayesian Models for Astrophysical Data, Cambridge Univ. Press
# (c) 2017,  Joseph M. Hilbe, Rafael S. de Souza and Emille E. O. Ishida 
# 
# you are kindly asked to include the complete citation if you used this 
# material in a publication

# Code 4.7 - Multivariate normal linear model in Python using Stan
# 1 response (y) and 2 explanatory variables (x1, x2)

# ^^ a implementação foi inspirada no autor acima

import numpy as np
import statsmodels.api as sm
import stan
from scipy.stats import uniform, norm
import json
# Data
np.random.seed(1056)                 # set seed to replicate example
nobs= X_train.shape[0]                         # number of obs in model 
x1 = X_train['1'].to_numpy()          # random uniform variable
x2 = X_train['2'].to_numpy()          # second explanatory
x3 = X_train['3'].to_numpy()
x4 = X_train['4'].to_numpy()

X = np.column_stack((x1,x2, x3, x4))        # create response matrix
# X = np.column_stack((x1, x2))
X = sm.add_constant(X)              # add intercept

y = y_train # agora faz regressão na abcissa do centro de massa calculado do corpo de prova

# Fit
data = {}                                # build data dictionary
data['nobs'] = nobs                      # sample size
data['x'] = X                            # explanatory variable         
data['y'] = y                            # response variable
data['k'] = data['x'].shape[1]       # number of explanatory variables

# data = json.dumps(list(data))

# data = data.to_json()


# STAN code
stan_code = """
data {
    int<lower=1> k;  
    int<lower=0> nobs;                                 
    matrix[nobs, k] x;                     
    vector[nobs] y;                     
}
parameters {
    matrix[k,1] beta;                                             
    real<lower=0> sigma;               
}
transformed parameters{
    matrix[nobs,1] mu;
    vector[nobs] mu2;

    mu = x * beta;
    mu2 = to_vector(mu);                 // normal distribution 
                                         // does not take matrices as input
}
model {
    for (i in 1:k){                      // Diffuse normal priors for predictors
        beta[i] ~ normal(0.0, 100);
    }
    sigma ~ gamma(76.083434761, 75.539775);           // A variância dos sensores nao é baixa mas eles são relativamente confiáveis
    
    y ~ normal(mu2, sigma);              // Likelihood function
}
"""

#  sigma ~ gamma(76.083434761, 75.539775);           




# Compile model
model = stan.build(stan_code, data)

# perform fit
fit2 = model.sample(num_chains=4, num_samples=1000)


# Output
nlines = 9                                   # number of lines in screen output

output = str(fit1).split('\n')
for item in output[:nlines]:
    print(item)   

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_00zm6ej4/model_llcrskhi.stan', line 24, column 32: Argument
    75.539775 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_00zm6ej4/model_llcrskhi.stan', line 24, column 18: Argument
    76.083434761 suggests there may be parameters that are not unit scale;
    consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_00zm6ej4/model_llcrskhi.stan', line 22, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   4% (303/8000)
Sampling:   9% (702/8000)
Sampling:  34% (2701/8000)
Sampling:  59% (4700/8000)
Sampling:  80% (6400/8000)
Sampling: 100% (8000/8000

<stan.Fit>
Parameters:
    beta: (5, 1)
    sigma: ()
    mu: (492, 1)
    mu2: (492,)
Draws: 4000


In [68]:
import arviz as a

summ = az.summary(fit2)
summ = summ[:][0:6]
summ

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"beta[0, 0]",18.141,1.291,15.653,20.525,0.019,0.013,4727.0,3208.0,1.0
"beta[1, 0]",21.907,4.995,12.651,31.030,0.069,0.050,5347.0,3301.0,1.0
"beta[2, 0]",-0.861,4.568,-9.725,7.496,0.068,0.063,4559.0,3281.0,1.0
"beta[3, 0]",0.522,96.894,-174.971,184.980,1.248,1.589,6047.0,3230.0,1.0
"beta[4, 0]",2.345,98.799,-182.660,179.315,1.358,1.567,5323.0,3168.0,1.0
sigma,27.096,0.324,26.483,27.698,0.004,0.003,5454.0,3441.0,1.0


In [106]:
coefsy = summ['mean'][0:6]
coefsy[0]

14.241

definindo funções para validação cruzada:

In [79]:
# avaliando erro quadrático médio

def EQM(c0, c1, c2, c3, c4, X_test, y_test):
    
    eqm = 0
    size = len(y_test)
    for i in range(size):
        
        eqm += (((c0 + c1*X_test['1'][i] + c2*X_test['2'][i] + c3*X_test['3'][i] + c4*X_test['4'][i]) - y_test[i])**2)/size
        
    return eqm 

In [83]:
EQM(coefsx[0], coefsx[1], coefsx[2], coefsx[3], coefsx[4], X_test, x_test)

16796.23254387031

In [84]:
EQM(coefsy[0], coefsy[1], coefsy[2], coefsy[3], coefsy[4], X_test, y_test)

1194.5757820429342

In [115]:
# avaliando quantos elementos estão dentro da regiao de confiança mutua

def conf(coefsx, coefsy, lamb, X_test, x_test, y_test):
     
    acertos = 0
    
    
    for i in range(len(y_test)): 
    
        x_prev = coefsx[0] + coefsx[1]*X_test['1'][i] + coefsx[2]*X_test['2'][i] + coefsx[3]*X_test['3'][i] + coefsx[4]*X_test['4'][i]
        y_prev = coefsy[0] + coefsy[1]*X_test['1'][i] + coefsy[2]*X_test['2'][i] + coefsy[3]*X_test['3'][i] + coefsy[4]*X_test['4'][i]
    
        if (x_test[i] > x_prev - lamb*coefsx[5] and x_test[i] < x_prev + lamb*coefsx[5] and
            y_test[i] > y_prev - lamb*coefsy[5] and y_test[i] < y_prev + lamb*coefsy[5]):

            acertos += 1
            
    return acertos/len(y_test)

In [125]:
lamb = 2

lambs = [0.1, 0.2, 0.5, 1, 1.5, 2]
acertos = [0]*len(lambs)

for i in range(len(lambs)):
    acertos[i] = conf(coefsx, coefsy, lambs[i], X_test, x_test, y_test)

acertos

[0.0,
 0.07109004739336493,
 0.7298578199052133,
 0.8246445497630331,
 0.8436018957345972,
 0.8957345971563981]